In [1]:
## This program predicts stock prices of 5 major US banks using linear regression models

In [2]:
import quandl
import numpy as np
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
import plotly.offline as pyo
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [3]:
#taking data from last 5 years of 5 major US banks

end= dt.datetime.now()
start= end - dt.timedelta(days=1825) 
start, end

(datetime.datetime(2019, 2, 8, 13, 28, 53, 882776),
 datetime.datetime(2024, 2, 7, 13, 28, 53, 882776))

In [4]:
# Taking the top 5 major banks in the US
# Bank names respectively: JPMorgan Chase, Bank of America, Wells Fargo, Citigroup,
# U.S. Bancorp

stocks= ['JPM', 'BAC', 'WFC', 'C','USB']
stocks

['JPM', 'BAC', 'WFC', 'C', 'USB']

In [18]:
# Getting the data from yahoo finance using pandas reader

df= yf.download(stocks, start, end)
df.tail()

[*********************100%%**********************]  5 of 5 completed


Price       Adj Close                                                   Close  \
Ticker            BAC          C         JPM        USB        WFC        BAC   
Date                                                                            
2024-01-31  34.009998  55.638958  174.360001  41.540001  49.830002  34.009998   
2024-02-01  33.549999  55.530003  173.729996  40.889999  48.730000  33.549999   
2024-02-02  33.470001  55.540001  174.729996  40.840000  49.160000  33.470001   
2024-02-05  32.990002  54.500000  174.500000  40.099998  48.700001  32.990002   
2024-02-06  33.040001  54.349998  175.100006  40.099998  48.290001  33.040001   

Price                                                    ...       Open  \
Ticker              C         JPM        USB        WFC  ...        BAC   
Date                                                     ...              
2024-01-31  56.169998  174.360001  41.540001  50.180000  ...  34.570000   
2024-02-01  56.060001  173.729996  40.889999  48.730000  ...  33.930000   
2024-02-02  55.540001  174.729996  40.840000  49.160000  ...  33.369999   
2024-02-05  54.500000  174.500000  40.099998  48.700001  ...  33.110001   
2024-02-06  54.349998  175.100006  40.099998  48.290001  ...  32.919998   

Price                                                      Volume            \
Ticker              C         JPM        USB        WFC       BAC         C   
Date                                                                          
2024-01-31  56.740002  176.199997  42.459999  51.009998  54716000  28967800   
2024-02-01  56.020000  173.639999  41.580002  49.820000  57084800  21178700   
2024-02-02  55.599998  173.300003  40.500000  48.500000  40401300  19269000   
2024-02-05  54.939999  173.860001  40.439999  48.610001  43573000  14040600   
2024-02-06  54.380001  174.610001  40.049999  48.520000  27188000  14832700   

Price                                     
Ticker           JPM       USB       WFC  
Date                                      
2024-01-31  11521800  13026100  20394400  
2024-02-01   9354800  16002800  27566100  
2024-02-02   8607600  11269300  19329000  
2024-02-05   7820200   8880300  14202700  
2024-02-06   6764800   8728400  13926600  

[5 rows x 30 columns]

In [6]:
df.index

DatetimeIndex(['2019-02-08', '2019-02-11', '2019-02-12', '2019-02-13',
               '2019-02-14', '2019-02-15', '2019-02-19', '2019-02-20',
               '2019-02-21', '2019-02-22',
               ...
               '2024-01-25', '2024-01-26', '2024-01-29', '2024-01-30',
               '2024-01-31', '2024-02-01', '2024-02-02', '2024-02-05',
               '2024-02-06', '2024-02-07'],
              dtype='datetime64[ns]', name='Date', length=1258, freq=None)

In [7]:
df.columns

MultiIndex([('Adj Close', 'BAC'),
            ('Adj Close',   'C'),
            ('Adj Close', 'JPM'),
            ('Adj Close', 'USB'),
            ('Adj Close', 'WFC'),
            (    'Close', 'BAC'),
            (    'Close',   'C'),
            (    'Close', 'JPM'),
            (    'Close', 'USB'),
            (    'Close', 'WFC'),
            (     'High', 'BAC'),
            (     'High',   'C'),
            (     'High', 'JPM'),
            (     'High', 'USB'),
            (     'High', 'WFC'),
            (      'Low', 'BAC'),
            (      'Low',   'C'),
            (      'Low', 'JPM'),
            (      'Low', 'USB'),
            (      'Low', 'WFC'),
            (     'Open', 'BAC'),
            (     'Open',   'C'),
            (     'Open', 'JPM'),
            (     'Open', 'USB'),
            (     'Open', 'WFC'),
            (   'Volume', 'BAC'),
            (   'Volume',   'C'),
            (   'Volume', 'JPM'),
            (   'Volume', 'USB'),
            ( 

In [8]:
# Filter out columns that are just the original close prices for each stock
close_prices_columns = [col for col in df.columns if col[0] == 'Close']
close_prices = df[close_prices_columns]

# Display the filtered close prices
print(close_prices.tail())


Price           Close                                             
Ticker            BAC          C         JPM        USB        WFC
Date                                                              
2024-02-01  33.549999  56.060001  173.729996  40.889999  48.730000
2024-02-02  33.470001  55.540001  174.729996  40.840000  49.160000
2024-02-05  32.990002  54.500000  174.500000  40.099998  48.700001
2024-02-06  33.040001  54.349998  175.100006  40.099998  48.290001
2024-02-07  33.209999  54.466999  175.289993  40.134998  48.540001


In [19]:
# Adjust the filter to select only columns that start exactly with 'Close_' and not 'Adj Close_'
# This assumes that after flattening, adjusted close columns start with 'Adj Close_' and you want to exclude them
close_prices_only = df_flattened[[col for col in df_flattened.columns if col.startswith('Close_') and not col.startswith('Adj Close_')]]

# Using Plotly to plot actual closing prices only
import plotly.graph_objs as go
from plotly.offline import iplot

# Create traces for each stock's actual closing prices
traces = [go.Scatter(
    x=close_prices_only.index, 
    y=close_prices_only[col], 
    mode='lines', 
    name=col.replace('Close_', '')  # Remove 'Close_' prefix for legend clarity
) for col in close_prices_only.columns]

# Define the layout of the plot
layout = go.Layout(
    title="Stock Closing Prices Over Time",
    xaxis=dict(title='Date'),
    yaxis=dict(title='Price (USD)'),
)

# Create the figure with data and layout
fig = go.Figure(data=traces, layout=layout)

# Plot the figure
iplot(fig)


In [10]:
# A variable for predicting 'n' days out into the future
forecast_out = 30

# Create a DataFrame to hold the close prices with predictions, ensuring it has a single-level column index
df_predictions = close_prices.copy()
df_predictions.columns = [col[1] for col in df_predictions.columns]  # Flatten the MultiIndex

# Adjust the loop to work with the flattened structure
for ticker in df_predictions.columns:
    # Create a prediction column for each ticker
    prediction_column_name = f'Prediction_{ticker}'  # This will create names like 'Prediction_BAC'
    df_predictions[prediction_column_name] = df_predictions[ticker].shift(-forecast_out)

# Display the first few rows to see the new structure with 'Prediction' columns
print(df_predictions.tail())


                  BAC          C         JPM        USB        WFC  \
Date                                                                 
2024-02-01  33.549999  56.060001  173.729996  40.889999  48.730000   
2024-02-02  33.470001  55.540001  174.729996  40.840000  49.160000   
2024-02-05  32.990002  54.500000  174.500000  40.099998  48.700001   
2024-02-06  33.040001  54.349998  175.100006  40.099998  48.290001   
2024-02-07  33.209999  54.466999  175.289993  40.134998  48.540001   

            Prediction_BAC  Prediction_C  Prediction_JPM  Prediction_USB  \
Date                                                                       
2024-02-01             NaN           NaN             NaN             NaN   
2024-02-02             NaN           NaN             NaN             NaN   
2024-02-05             NaN           NaN             NaN             NaN   
2024-02-06             NaN           NaN             NaN             NaN   
2024-02-07             NaN           NaN             

In [11]:
### Creating a independent data set (X)

# Drop the prediction columns to keep only the close prices
X = df_predictions.drop([col for col in df_predictions.columns if 'Prediction' in col], axis=1)

# Convert the DataFrame to a NumPy array
X = np.array(X)

# Remove the last 'forecast_out' rows to align with the number of prediction data points
X = X[:-forecast_out]

# Display the array
print(X)

[[ 28.29000092  62.00999832 101.36000061  50.58000183  47.65000153]
 [ 28.40999985  61.61000061 100.87999725  50.54999924  47.65000153]
 [ 28.69000053  62.66999817 102.59999847  51.20999908  49.04999924]
 ...
 [ 32.97999954  50.06999969 166.55000305  42.45999908  49.34000015]
 [ 33.20000076  50.59999847 167.5         43.31000137  49.45000076]
 [ 33.43000031  50.86000061 167.3999939   43.50999832  49.18000031]]


In [12]:
### Creating a dependent data set (Y)

# Choose a specific stock ticker for which you want to create the dependent data set
ticker = 'JPM'  # Replace with the ticker you are interested in

# Extract the prediction column for the chosen ticker
Y = np.array(df_predictions[f'Prediction_{ticker}'])

#Get all of the values of y except the last 'n' rows
Y= Y[:-forecast_out]

# Display the array
print(Y)

[ 98.93000031  99.91999817  99.58000183 ... 174.5        175.1000061
 175.28999329]


In [13]:
# Split the data into 80% training and 20% testing 
X_train, X_test, Y_train, Y_test= train_test_split(X,Y, test_size=0.2)

In [14]:
# Create and train the linear regression model
lr= LinearRegression()
# Train the model
lr.fit(X_train, Y_train)

LinearRegression()

In [15]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction
# The best possible score in 1.0
lr_confidence= lr.score(X_test, Y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.7370535946168576


In [22]:
# last_known_date = pd.to_datetime('2024-02-07')
# future_dates = pd.date_range(start=last_known_date, periods=forecast_out + 1, freq='B')[1:]  # Exclude the start date
# X_future = np.tile(X[-1], (len(future_dates), 1))

# # Make predictions
# forecasted_prices = lr.predict(X_future)

# # Associate predictions with their corresponding future dates
# future_predictions = pd.DataFrame(forecasted_prices, index=future_dates, columns=['Predicted Price'])

# # Print the predictions
# print(future_predictions)

last_known_date = pd.to_datetime('2024-02-07')
future_dates = pd.date_range(start=last_known_date, periods=forecast_out + 1, freq='B')[1:]  # Exclude the start date

# Initialize the array for the first future prediction with the last known historical data
last_features = X[-1].reshape(1, -1)

# Initialize a list to store the predictions
forecasted_prices_list = []

for _ in future_dates:
    # Predict the next day's price using the most recent features
    next_day_prediction = lr.predict(last_features)
    
    # Append the prediction to the list
    forecasted_prices_list.append(next_day_prediction[0])
    
    # Update the features for the next prediction
    # Here, we simulate 'rolling' the features to make space for the new prediction
    # This assumes the last value in 'last_features' is the most recent closing price used for prediction
    # If your model uses additional or different features, adjust this update logic accordingly
    if last_features.shape[1] > 1:
        # Shift features to the left if there are multiple features, and set the last one as the new prediction
        last_features = np.roll(last_features, -1)
        last_features[0, -1] = next_day_prediction
    else:
        # If there's only one feature, just update it to the new prediction
        last_features = next_day_prediction.reshape(1, -1)

# Create a DataFrame for the predictions with the corresponding future dates
future_predictions = pd.DataFrame(forecasted_prices_list, index=future_dates, columns=['Predicted Price'])

# Print the predictions
print(future_predictions)


            Predicted Price
2024-02-08       173.710421
2024-02-09      -116.364489
2024-02-12        33.197223
2024-02-13         5.419413
2024-02-14      -263.569387
2024-02-15       -19.778314
2024-02-16       -83.120070
2024-02-19      -340.075514
2024-02-20       194.301532
2024-02-21      -102.047480
2024-02-22       -68.387846
2024-02-23       508.435800
2024-02-26        25.103189
2024-02-27       231.690640
2024-02-28       774.948232
2024-02-29      -206.188926
2024-03-01       419.333782
2024-03-04       413.120492
2024-03-05      -761.148712
2024-03-06       221.522712
2024-03-07      -296.838464
2024-03-08     -1436.296470
2024-03-11       375.743128
2024-03-12      -978.780582
2024-03-13     -1109.892848
2024-03-14      1193.124439
2024-03-15      -862.312463
2024-03-18       275.092068
2024-03-19      2605.781654
2024-03-20      -799.001298


In [17]:
import plotly.graph_objs as go
import pandas as pd

# Ensure 'ticker' is set to the correct column name as verified
ticker = 'JPM'  #Change accordingly of the stock you want

# Create a date range for the next 30 days starting from the last date in your dataset
last_date = df_predictions.index[-forecast_out]
date_range = pd.date_range(last_date, periods=forecast_out)

# Predict the closing prices for the next 30 days
X_lately = X[-forecast_out:]
predicted_prices = lr.predict(X_lately)

# Plot the actual closing prices
actual_trace = go.Scatter(
    x=df_predictions.index,
    y=df_predictions[ticker],  # Adjusted to use ticker directly if column names are flattened
    mode='lines',
    name='Actual Prices'
)

# Plot the predicted closing prices
predicted_trace = go.Scatter(
    x=date_range,
    y=predicted_prices,
    mode='lines+markers',
    name='Predicted Prices',
    line=dict(dash='dash')
)

# Combine the traces
data = [actual_trace, predicted_trace]

# Set up the layout
layout = go.Layout(
    title=f'Actual vs Predicted Closing Prices for {ticker}',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Price (USD)'),
)

# Create the figure with data and layout
fig = go.Figure(data=data, layout=layout)

# Show the figure
fig.show()
